<a href="https://colab.research.google.com/github/shitote/dist-repo/blob/main/introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NLP are also refered to as a sequence to sequence problems.(seq2seq)

In [ ]:
!nvidia-smi -L

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# import helper functions.
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset
Kaggle's introduction to NLP dataset (text samples of Tweets labbelled as disaster or not disarster)

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the data
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset.

Read the text file in the notebook

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

In [ ]:
train_df["text"][1]

In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

In [ ]:
# Note that the test data frame have no target values.
test_df.head()

In [ ]:
# Find Out if the dataset is baklanced or not?
train_df.target.value_counts()

In [ ]:
# Find the number of total samples.
len(train_df), len(test_df)

In [ ]:
# visualize random training examples.
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+10].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(Not real disaster)")
  print(f"Text:\n{text}\n")
  print("-----\n")

### Split the data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

In [ ]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

In [ ]:
train_sentences[:10], train_labels[:10]

## Coverting text to numbers
* Tokenization - This is direct mapping of token to number
* Emmbedding - Create a matrix of feature vectors for each token (
    the size fo the feature vector can be defined for each token embedding can be learned
)

### Text vectorization (tokenization)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters.
text_vectorizer = TextVectorization(max_tokens=None,  # How many words in the vocabulary (automaticaly add <oov>)
                                    standardize='lower_and_strip_punctuation',  # Remove the fluf
                                    split='whitespace',
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False
                                    )

In [ ]:
len(train_sentences[0].split())

In [ ]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

In [17]:
# set text vectorization variables
max_vocab_length = 1000
max_length = 16

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [18]:
# Fit the text vectorizer to the trainin text
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create a sample sentence and tokenize it
sample_sentence = "There are floods everywhere"
text_vectorizer([sample_sentence])

In [ ]:
random_sentence = random.choice(train_sentences)
print(f'Original text: \n {random_sentence}\n\n Vectorized version:')
text_vectorizer([random_sentence])

In [ ]:
# et the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words i vocab: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

In [22]:
### Create an Embedding using an embedding layer
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,  # set the input shape
                            output_dim=128,  # output shape
                            input_length=max_length # how long is each input
                            )

embedding


In [ ]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence} \
      \n\nEmbedding version: ")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

## Modelling a text dataset (running a serries of experiments)

The data is now in numbers it is time to build a series of modelling experiments.

* model 0: naive bayes (baseline), this is from sklearn ML
* model 1: Feed-forward neural network (dence model)
* model 2: LSTM model (RNN)
* model 3: GRU model (RNN)
* model 4: Bidirectional Neural Networks(RNN)
* model 5: !D Convolutional neural nerworks (CNN)
* model 6: Tensoflow hub Pretrained Feature extraction (using transfer learnin NLP)
* model 7: Same as model 6 with 10% of training data.

THs use od standard steps of modelling in tensorflow:
* Crreate a model.
* Build a model.
* FIt the model.
* Evaluate the model.

### Model 0: Getting a baseline.

**NOte** It is a better proctice to use non-DL alorithms to create the baseline model since they are fast and easy to train then latter impliment it with the DL to improve the performance.

In [ ]:
# Trun the data into numerical.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create a tokenization and modellin pipeline.
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),   #Convert words to numbers.
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

In [ ]:
# Baseline model evaluation.
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline model score: {baseline_score*100:.2f}%")

In [ ]:
# Make predictions.
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

In [ ]:
train_labels

In [28]:
# Fuction to evalate: accuracy, precision, recall, f1_score.
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results


In [ ]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

### Model_1: A simple dence model

In [30]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

SAVE_DIR = 'model_logs'

In [31]:
# Build a model using the functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)  # Input are 1-dimensional array of string.
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [ ]:
model_1.summary()

In [33]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
train_sentences.shape, train_labels.shape

In [35]:
train_labels = tf.reshape(train_labels, (-1, 1))
val_labels = tf.reshape(val_labels, (-1, 1))

In [ ]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name='model_1_dence')])

In [ ]:
model_1.evaluate(val_sentences, val_labels)

In [ ]:
# Make somce Predictions and evaluate them
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

In [ ]:
model_1_pred_probs[0]

In [ ]:
# Convert the models prediction into probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

In [ ]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

In [ ]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

## Visualize the learned embedding.

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

In [ ]:
model_1.summary()

In [ ]:
# Get the wight metrix of embeddin layer
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

### Create embedding files.

In [46]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + '\n')
  out_m.write(word + '\n')
out_v.close()
out_m.close()

In [ ]:
# Dowload file from colab to upload to the projector
# word to vector jay alama
try:
  from google.colab import files
  files.download("vectors.tsv")
  files.download('metadata.tsv')
except Exception:
  pass

## Recurrent Neural Networks (RNN)

RNN are useful for sequence data.

The premise of a recurrent neuaral network is to use the representatoin of a previous input to aid the representation of a later input

The internals of RNN:
- MIT's sequence modelling.
- Chris olah's intor to LSTMs:
- Andrej Kapathy's the unreasonable effectiveness of recurrent neural networks.

### Model 2: LSTM
LSTM = Long short term memory (One of the most popular LSTM cells)
* The structure of an RNN trypically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probabilities)

In [48]:
# Tenh
# Create an LSTM model.
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name='model_2_LSTM')

In [49]:
# Compile the model.
model_2.compile(loss='binary_crossentropy',
  optimizer=tf.keras.optimizers.Adam(),
  metrics=["accuracy"])

In [ ]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 'model_2_LSTM')])

In [ ]:
# Make predictions with the LSTM model.
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

In [ ]:
# Calculate model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)
model_2_results

### Model 3: GRU

This is also a popular RNN component GRU - gated recurrent Unit

The GRU have the same features to an LSTM cell but have less parameters,

RFewources:
understantin the GRU networks by simion


In [54]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x)
# x = layers.LSTM(64)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [ ]:
model_3.summary()

In [56]:
# Compile the model
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_3_GRU')])

In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

In [ ]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

In [ ]:
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (the same way you will read a kiswahili sentence).
The bidirection RNN go from right to let as well as from  left to right

In [61]:
# Build a bidirection RNN
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_4_bidirectional')

In [ ]:
model_4.summary()

In [63]:
model_4.compile(loss="binary_crossentropy",
                optimizer= tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_4_bidirectional')])

In [ ]:
# make predictions.
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

In [ ]:
# Turn the results into a comparable features.
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

In [ ]:
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

## Convolutional Neural Networks for text and othet types od seu=quences

The typical structure of a convolutional neural network

Input (text) -> Tokenization -> Embedding -> Layers (typically conv1D + pooling) -> Outputs (class probabilities)

### Model 5: conv1D

In [ ]:
# Make a test of the embedding layer, conv1D layer and max pooling layers
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation='relu',
                        padding='valid')
conv_1d_output = conv_1d(embedding_test)  # pass the test embeddin into the conv1d layer.
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)   # get the most important fearutes.

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

In [ ]:
embedding_test

In [ ]:
conv_1d_output[:2]

In [ ]:
max_pool_output

In [ ]:
# Create d dimensional convolutional neural network layer to model sequences.
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation='relu', padding= 'valid')(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name='model_5_conv1D')

# Compile the model and fit it ata one go
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_5.summary()

In [73]:
train_sentences_tensor = tf.convert_to_tensor(train_sentences)
val_sentences_tensor = tf.convert_to_tensor(val_sentences)

In [ ]:
# fit model 5
model_5_history = model_5.fit(train_sentences_tensor,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences_tensor, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'conv1D')])


In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:5]

In [ ]:
# Make the results comparerable.
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

 ## # Model 6: Tensorflow Hub Pretrained Sentence Encoder

In [77]:
# import tensorflow_hub as hub
# import tensorflow as tf

# english_sentences = tf.constant(["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."])

# preprocessor = hub.KerasLayer(
#     "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3")
# encoder = hub.KerasLayer(
#     "https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/cmlm-en-base/1")

# english_embeds = encoder(preprocessor(english_sentences))["default"]

# print (english_embeds)


In [78]:
# !pip install tensorflow-text

In [79]:
# !pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf

print(tf.__version__)

from json import encoder
## Model 6: Tensorflow hub pretrained sentence encoder
Transfer learning for NLP, Specificaly use tensorflow hub universal sentence encoder

In [81]:
import tensorflow_hub as hub

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
embed = model
embed_samples = embed([sample_sentence,
                       "When you add the universal sentences encoder to a sentence, it converts it to numbers"])
print(embed_samples[0][:50])


In [ ]:
embed_samples

In [84]:
# Create a keras layer using pretrained layer from Hub
sentence_encoder_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder/4",
    input_shape=[],
    dtype=tf.string,
    trainable=False,
    name="USE"
    )

In [85]:
# Create the model using sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
], name='model_6_USE')

In [ ]:
# Compile the model
model_6.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_6.summary()

In [ ]:
# TRain a classifier on top of USE embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'tf_hub_sentence_encoder')])

In [ ]:
# make predictions with USE TF HUb model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

In [ ]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

In [ ]:
# Calculate the model 6 performance metrics
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

In [91]:
# improve the preformance of the  model
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
], name='model_7_USE')

In [ ]:
# Compile the model.
model_7.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_7.summary()

In [ ]:
# Fit the model.
model_7_history = model_7.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     'model_7_USE')])

In [ ]:
# Make predictions for the model.
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:5]

In [ ]:
# make the prediction result comparerable.
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:5]

In [ ]:
# calculate model 7 presormance metrics.
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)
model_7_results

## Model 7: TF HUB Pretrained USE with only 10% of the training data

In [ ]:
# create subsets of 10% of training data
train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)
train_10_percent.head(), len(train_10_percent), len(train_df_shuffled)

In [98]:
# This is an element of data leakages.
train_sentences_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [99]:
# Solve the proble of data leakage.
train_10_percent_split = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]
len(train_sentences_10_percent), len(train_labels_10_percent)

(685, 685)

In [ ]:
train_10_percent[:10]

In [ ]:
pd.Series(np.array(train_labels_10_percent).ravel()).value_counts()

In [ ]:
train_sentences_10_percent[:5]

#### Clone model 6 to adapt it to model 7.

In [ ]:
model_7 = tf.keras.models.clone_model(model_6)

# compile the model.
model_7.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_7.summary()

In [ ]:
# Fit the model to the data
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                    'tf_hub_sentence_encoder_10_percent_correct')])

In [ ]:
# Make prediction of the models that is trained on 10 percent of the data.
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:5]

In [ ]:
# Turn the pred probs into labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

In [ ]:
# Model 7 predictions evaluations.
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)
model_7_results

## Compare the performance of all the models

In [ ]:
# combine model results into a dataframe
all_model_results = pd.DataFrame({"0_baseline": baseline_results,
                                  "1_simple_dense": model_1_results,
                                  "2_lstm": model_2_results,
                                  "3_gru": model_3_results,
                                  "4_bidirectional": model_4_results,
                                  "5_conv1d": model_5_results,
                                  "6_tf_hub_USE": model_6_results,
                                  "7_tf_hub_USE_10%": model_7_results
                                  })
all_model_results = all_model_results.transpose()
all_model_results

In [ ]:
# Reduce the accuracy to the same scale as other metrics
all_model_results['accuracy'] = all_model_results['accuracy']/100
all_model_results

In [ ]:
# prompt: Using dataframe all_model_results: scatter

import altair as alt
alt.Chart(all_model_results).mark_circle().encode(
    x='accuracy',
    y='precision',
    color=alt.Color('f1', scale=alt.Scale(scheme='viridis'))
).interactive()


In [ ]:
# Ploa and compare all of the model results
all_model_results.plot(kind='bar', figsize=(10, 7)).legend(bbox_to_anchor=(1.0, 1.0))

In [ ]:
# Sort model results by f1_score.
all_model_results.sort_values("f1", ascending=False)['f1'].plot(kind='bar', figsize=(10, 7))

📖**Resource:** checkout weihts and bieses for experiment tracking

### Save and load the model

There are two main formats to save the model to in TensorFlow:
1. The HDF4 format,
2. The `SavedModel` format

In [ ]:
model_6_results

In [ ]:
# save TF hub SE model to HDF5
model_6.save('model_6.h5')

In [115]:
# Load model with custom hub layer (reauored by HDF5)
import tensorflow_hub as hub
loaded_model_6 = tf.keras.models.load_model('model_6.h5',
                                            custom_objects={'KerasLayer': hub.KerasLayer})

In [ ]:
# Evaluat the loaded model.
loaded_model_6.evaluate(val_sentences, val_labels)

In [117]:
# use the saved model format
model_6.save("model_6_savedModel_format")

In [118]:
load_model_6_savedModel = tf.keras.models.load_model('model_6_savedModel_format')

In [ ]:
# Make the evaliations of the save model.
load_model_6_savedModel.evaluate(val_sentences, val_labels)

## Find the most wrong examples

* If the model still isn't perfect, what examples is it getting wrong?
* And of those wrong examples which one is it getting *most* wron (Those with the prediction probabilities closser to the opposite class)

**This is call data driven data exploration or active learning**

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
!unzip 08_model_6_USE_feature_extractor.zip

In [ ]:
model_6_pretrained = tf.keras.models.load_model('08_model_6_USE_feature_extractor')
model_6_pretrained.evaluate(val_sentences, val_labels)

In [ ]:
# Make Predictions with the loaded model
model_6_pretrained_pred_probs = model_6_pretrained.predict(val_sentences)
model_6_pretrained_preds = tf.squeeze(tf.round(model_6_pretrained_pred_probs))
model_6_pretrained_preds[:10]

In [ ]:
model_6_pretrained_pred_probs = tf.squeeze(model_6_pretrained_pred_probs)
model_6_pretrained_pred_probs.shape

In [ ]:
# Create the DataFrame
val_df = pd.DataFrame({"text": val_sentences,
                       "target": tf.squeeze(tf.round(val_labels)),
                       "pred": model_6_pretrained_preds,
                       "pred_prob": model_6_pretrained_pred_probs})
val_df.head()

In [ ]:
# Find the wrong prodictions and sort by prediction probabilities.
most_wrong = val_df[val_df['target'] != val_df['pred']].sort_values('pred_prob', ascending=False)
most_wrong[:10]

In [ ]:
# Check the false positives
for row in most_wrong[:10].itertuples():
  _, text, target, pred, pred_prob = row
  print(f"Target: {target}, Pred: {pred} Prob: {pred_prob}")
  print(f"Text:\n{text}\n")
  print("-------------------\n")

In [ ]:
# Check the false neatives
for row in most_wrong[-10:].itertuples():
  _, text, target, pred, pred_prob = row
  print(f"Target: {target}, Pred: {pred}, Prob: {pred_prob}")
  print(f"Text: \n {text}")
  print("-------------\n")

In [ ]:
test_df.head()

 ## Makin prediction on the test dataset.

In [ ]:
 test_sentences = test_df['text'].to_list()
 test_sentences[:6]

In [ ]:
# Making predictions on the test dataset and visualizing them.
test_sentences = test_df['text'].to_list()
test_samples = random.sample(test_sentences, 10)
for test_sample in test_samples:
  pred_prob = tf.squeeze(model_6_pretrained.predict([test_sample]))  # the model expets a list.
  pred = tf.round(pred_prob)
  print(f"Pred: {int(pred)}, Prob: {pred_prob}")
  print(f"Text: \n {test_sample}")
  print("---------\n")

## The speed/score tradeoff

In [164]:
import time
def pred_timer(model, samples):
  """
  Times how long a model takes to make proedictions on samples.
  """
  start_time = time.perf_counter()  # ge start time
  model.predict(samples)  # make predictions
  end_time = time.perf_counter()  # et finish time
  total_time = end_time - start_time
  time_per_pred = total_time/len(samples)
  return  total_time, time_per_pred

In [165]:
model_6_total_pred_time, model_6_time_per_pred = pred_timer(
    model=model_6_pretrained,
    samples=val_sentences)
model_6_total_pred_time, model_6_time_per_pred

24/24 [==============================] - 0s 8ms/step


(0.27332212800047273, 0.0003586904566935338)

In [ ]:
# Baseline model times per pred
baseline_total_pred_time, baseline_time_per_pred = pred_timer(
    model=model_0,
    samples=val_sentences)
baseline_total_pred_time, baseline_time_per_pred

In [ ]:
model_6_pretrained_results = calculate_results(y_true=val_labels,
                                               y_pred=model_6_pretrained_preds)
model_6_pretrained_results

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(11, 9))
plt.scatter(baseline_time_per_pred, baseline_results["f1"], label='baseline')
plt.scatter(model_6_time_per_pred, model_6_pretrained_results['f1'], label='tf_hub_sentence_encoder')
plt.legend()
plt.title("F1 score versus time per prediction")
plt.xlabel("Time per prediction")
plt.ylabel("F1-score")